In [111]:
!pip install opencv-python

In [112]:
# Pra rodar em casa
import sys
sys.path.append("/home/pedro/miniconda3/lib/python3.9/site-packages")

In [205]:
from sklearn.preprocessing import normalize
import numpy as np
from scipy import signal


In [176]:
import cv2

image = cv2.imread('img.jpg', 0)
h, w = image.shape
print (h, w)

for i in range(h):
    for j in range(w):
        if( image[i][j] > 128):
            print ('.', end='')
        else:
            print ('1', end='')
    print('\n', end='')



40 42
........................1111111111........
...................1111111111111111.......
..................111111111111111111......
................111111111111111111111.....
...............11111111111111111111111111.
.............11111111111........1111111111
............1111111111..........1111111111
............11111111...........1111111111.
...........11111111.........111111111111..
...........111111........111111111111.....
...........111111........11111111111......
...........11111......111111111111........
..........111111.....1111111111...........
..........111111...1111111111.............
..........1111111.1111111111..............
..........111111111111111.................
...........1111111111111..................
...........11111111111....................
...........1111111111.....................
...........11111111.......................
..........111111111.......................
..........111111111.......................
........11111111111.......................
.....

In [132]:
def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return cv2.resize(output, (40, 40))

In [202]:
def gaussian_kernel(n, std, normalised=False):
    '''
    Generates a n x n matrix with a centered gaussian 
    of standard deviation std centered on it. If normalised,
    its volume equals 1.'''
    gaussian1D = signal.gaussian(n, std)
    gaussian2D = np.outer(gaussian1D, gaussian1D)
    if normalised:
        gaussian2D /= (2*np.pi*(std**2))
    return gaussian2D

In [206]:
gaussian = gaussian_kernel(5, 1)

In [208]:
kernel_1 = [[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]
kernel_2 = [[-1, -1, -1], [0, 0, 0], [1, 1, 1]]
kernel_3 = [[1, 1, 1], [0, 0, 0], [-1, -1, -1]]
kernel_4 = [[1, 0, -1], [1, 0, -1], [1, 0, -1]]
kernel_5 = [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
kernel_6 = gaussian

In [209]:
image_conv = convolve2D(image, kernel_6)

In [210]:
h, w = image_conv.shape
print (h, w)

for i in range(h):
    for j in range(w):
        if( image_conv[i][j] > 128):
            print ('.', end='')
        else:
            print ('1', end='')
    print('\n', end='')

40 40
...................111111111111111......
..................1111111111111111......
.................11111..........111.....
...............11111.............1111...
..............11111..............1111111
.............1111................111111.
............1111................11111...
...........1111................1111.....
...........111...............11111......
...........111............111111........
...........11............1111...........
...........11..........11111............
..........111........11111..............
..........1111......1111................
..........1111....1111..................
...........11111.1111...................
...........111111111....................
...........1111111......................
...........111111.......................
...........111111.......................
..........111111........................
..........111111........................
.........1111111........................
........11111111........................
.......111

In [155]:
import matplotlib.pyplot as plt

In [156]:
def horizontal_histogram(image):
    h, w = image.shape
    hist = [0] * h
    for i in range(h):
        for j in range(w):
            if image[i][j] <= 128:
                hist[i] += 1
    return hist

In [157]:
horizontal_histogram(image)

[10,
 16,
 18,
 21,
 26,
 21,
 20,
 18,
 20,
 18,
 17,
 17,
 16,
 16,
 17,
 15,
 13,
 11,
 10,
 8,
 9,
 9,
 11,
 12,
 12,
 13,
 12,
 13,
 13,
 13,
 13,
 12,
 11,
 12,
 13,
 16,
 16,
 13,
 12,
 8]

In [159]:
def vertical_histogram(image):
    h, w = image.shape
    hist = [0] * w
    for i in range(w):
        for j in range(h):
            if image[j][i] <= 128:
                hist[i] += 1
    return hist

In [214]:
f = open("digits/files.txt", "r")
count = 0
features = ""
i = 0
numeros = [0] * 10
for x in f:
    image = cv2.imread("digits/" + x.split()[0], 0)
    image = cv2.resize(image, (40, 40))
    #conv_1 = convolve2D(image, kernel_1)
    #conv_2 = convolve2D(image, kernel_2)
    #conv_3 = convolve2D(image, kernel_3)
    #conv_4 = convolve2D(image, kernel_4)
    #conv_5 = convolve2D(image, kernel_5)
    #conv_6 = convolve2D(image, kernel_6)
    h, w = image.shape
    
    # histograms
    #features += ' '.join(str(e) for e in horizontal_histogram(image)) + ' '
    #features += ' '.join(str(e) for e in vertical_histogram(image)) + ' '
    
    # convolutions
    #for ele in conv_1:
    #    features += ' '.join(str(e) for e in ele) + ' '
    #for ele in conv_2:
    #    features += ' '.join(str(e) for e in ele) + ' '
    #for ele in conv_3:
    #    features += ' '.join(str(e) for e in ele) + ' '
    #for ele in conv_4:
    #    features += ' '.join(str(e) for e in ele) + ' '
    #for ele in conv_5:
    #    features += ' '.join(str(e) for e in ele) + ' '
    #for ele in conv_6:
    #    features += ' '.join(str(e) for e in ele) + ' '
    
    for ele in image:
        features += ' '.join(str(e) for e in ele) + ' '
    
    features += str(x.split()[1])
    features += '\n'
    numeros[int(x.split()[1])] += 1
    #    for i in range(h):
    #        for j in range(w):
    #            if( image[i][j] > 128):
    #                print ('.', end='')
    #            else:
    #                print ('1', end='')
    #        print('\n', end='')
    count += 1
    #print(x.split()[0])
    #print(x.split()[1])


In [215]:
features[0:50000]

'255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 254 254 254 254 254 254 254 255 249 233 177 3 44 239 254 255 255 253 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 253 253 253 253 253 254 254 254 236 140 3 3 0 2 32 204 254 254 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 254 253 253 253 253 252 253 223 110 0 1 1 1 1 2 35 255 254 255 253 254 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 254 254 254 253 253 254 254 255 254 168 1 3 1 1 2 0 0 17 136 254 249 253 254 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 253 254 252 251 188 84 56 1 2 2 1 3 2 0 0 28 253 252 255 252 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 254 254 253 253 254 162 45 29 0 2 3 1 2 0 1 0 0 2 6 63 253 255 253 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 253 253 134 6 6 3 0 1 1 0 0 0 1 1 0 0 2 0 1 22 255 

In [216]:
file = open ("features.txt", "w")
file.write(features)
file.close()

In [58]:
numeros

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]